In [74]:
import itertools #iteration tools
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # nice plots
import matplotlib.pyplot as plt # plots
from ipy_table import *

#preprocessing
from sklearn.preprocessing import LabelEncoder

# machine learning
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# Import and suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [75]:
##CHECK THOSEOUT
from imblearn.over_sampling import SMOTE
#import xgboost

### Get train Data

In [76]:
train_df = pd.read_csv(r'C:/Users/ernest.chocholowski/Desktop/Datasets/Titanic/train.csv')
test_df = pd.read_csv(r'C:/Users/ernest.chocholowski/Desktop/Datasets/Titanic/test.csv')
both = [train_df, test_df]
train_df.head()
super_table=[]
super_table.append(['name', 'reg_function', 'trainset_acc', 'testset_acc', 'r2score', 'conf_matrx', 'kaggle_acc'])
df_load = pd.read_csv(r'C:/Users/ernest.chocholowski/Desktop/GIT/Titanic/table_eCh.csv')
for row in df_load.values.tolist():
    super_table.append(row)

In [79]:
for df in both:
    df['Title'] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    df['Title'] = df['Title'].replace(['Lady', 'Countess', 'Don', 'Dr', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    df['Title'] = df['Title'].replace(['Capt', 'Col', 'Major'], 'Officer')
    df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
    df['Title'] = df['Title'].replace('Mme', 'Mrs')

In [80]:
#labelling
var_mod = ['Title', 'Sex', 'Ticket', 'Embarked']
le = LabelEncoder()
for i in var_mod:
    mask = ~train_df[i].isnull()
    train_df[i][mask] = le.fit_transform(train_df[i][mask])
    mask2 = ~test_df[i].isnull()
    test_df[i][mask2] = le.fit_transform(test_df[i][mask2])
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
Title           object
dtype: object

In [84]:
train_df = train_df.drop(['Cabin', 'Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Cabin', 'Name'], axis=1)

In [81]:
#Probabilities of each age in dataset :>
#kek=train_df.groupby(['Sex', 'Pclass', 'Title']).Age.apply(lambda g: g.value_counts()/len(g))

In [82]:
#perform imputation!
#imputation of age with imputer
from sklearn.preprocessing import Imputer
imp=Imputer(missing_values="NaN", strategy="mean" )
train_df["Age"]=imp.fit_transform(train_df[["Age"]]).ravel()
test_df["Age"]=imp.fit_transform(test_df[["Age"]]).ravel()

#impute embarked as the most frequent one
freq_port = train_df.Embarked.dropna().mode()[0]
train_df['Embarked'] = train_df['Embarked'].fillna(freq_port)

#impute missing Fare value
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)

In [85]:
train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Title
0,0,3,1,22.0,1,0,523,7.2500,2,2
1,1,1,0,38.0,1,0,596,71.2833,0,3
2,1,3,0,26.0,0,0,669,7.9250,2,1
3,1,1,0,35.0,1,0,49,53.1000,2,3
4,0,3,1,35.0,0,0,472,8.0500,2,2


In [204]:
# Import statements required for Plotly 
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
data = [go.Bar(
            x=train_df["Survived"].value_counts().index.values,
            y= train_df["Survived"].value_counts().values
    )]

py.iplot(data, filename='basic-bar')

In [185]:
#Split data for train and test

#set target for training
target = train_df["Survived"]
target.head(3)

# Import the train_test_split method
from sklearn.model_selection import train_test_split
# Split data into train (2/3rd of data) and test (1/3rd of data)
train, test, target_train, target_val = train_test_split(train_df, target, train_size= 0.66, random_state=0);

In [192]:
def regr_equation(logreg, train, target):
    if type(model) is LogisticRegression:
        coef = logreg.coef_[0]
        intercept = "{:.2f}".format(logreg.intercept_[0])
    else:
        coef = logreg.coef_
        intercept = "{:.2f}".format(logreg.intercept_)
        
    output = target.name + ' = ' + str(intercept) + ' + '
    coeff_df = pd.DataFrame(train.columns.delete(0))
    coeff_df.columns = ['Feature']
    coeff_df["Correlation"] = pd.Series(logreg.coef_[0])
    features = coeff_df['Feature'].tolist()
    coefficients = coeff_df['Correlation'].tolist()
    
    for coeff, feature in zip(coefficients, features):
        coeff_str = "{:.2f}".format(coeff)
        output += coeff_str + "*" + str(feature) + " + "
    return output[:-3]

In [193]:
def R2_score(model, X, y, sample_weight=None):
    from sklearn.metrics import r2_score
    return r2_score(y, model.predict(X), sample_weight=sample_weight)

In [194]:
def confusion_matrix(model, X, y):
    from sklearn.metrics import confusion_matrix
    return confusion_matrix(y, model.predict(X))

In [195]:
def test_model (model, train, test, target_train, target_val, submission_name = None):

    model.fit(train, target_train)

    #Calc parameters
    trainset_acc = round(model.score(train, target_train) * 100, 2)
    testset_acc = round(model.score(test, target_val) * 100, 2)
    if type(model) is LogisticRegression:
        function_str = regr_equation(model, train, target_train)
    elif type(model) is LinearRegression:
        function_str = regr_equation(model, train, target_train)
    else :
        function_str = "NA"    
    
    r2_score = R2_score(model, train, target_train)
    if type(model) is not LinearRegression:
        conf_matrix = confusion_matrix(logreg,train,target_train)
        conf_matrix = 'TN: '+str(conf_matrix[0][0])+', FP: '+str(conf_matrix[0][1])+ \
                          ', FN: '+str(conf_matrix[1][0])+', TP: '+str(conf_matrix[1][1])
    else:
        conf_matrix = "NA"
    kaggle = "not_tested"
    
    #prints
    print('Submission name:', submission_name )
    print('Regression function:\n', function_str)
    print('Accuracy on train set:', trainset_acc)
    print('Accuracy on test set:', testset_acc)
    print("R2 score:", r2_score)
    print("Confusion matrix:\n", conf_matrix)

    
    return [submission_name, function_str, str(trainset_acc), str(testset_acc), r2_score, conf_matrix, kaggle]

In [196]:
model = LinearRegression()

In [197]:
output = test_model(model, train, test, target_train, target_val, submission_name='logisticReg_basicImputator_Title')

Submission name: logisticReg_basicImputator_Title
Regression function:
 Survived = -0.00 + 1.00*Pclass + nan*Sex + nan*Age + nan*SibSp + nan*Parch + nan*Ticket + nan*Fare + nan*Embarked + nan*Title
Accuracy on train set: 100.0
Accuracy on test set: 100.0
R2 score: 1.0
Confusion matrix:
 NA


In [39]:
#Setup
x_selection = train_df.columns.values
x_selection = np.delete(x_selection, [0,1]) ##ONLY IF "SURVIVED" and "PASSENGERID" are in the selection!!
suffix = '_basicImputator_Title'

modelSVC = SVC()
modellinSVC = LinearSVC()
modelKN = KNeighborsClassifier(n_neighbors = 3)
modelGNB = GaussianNB()
modelPercp = Perceptron()
modelSGD = SGDClassifier()
modelTree = DecisionTreeClassifier()
modelRndForest = RandomForestClassifier(n_estimators=100)

models = {modelSVC: "SVC", modellinSVC: "LinSVC",
          modelKN: "KN", modelGNB: "GaussianNB",
          modelPercp: "Percep", modelSGD: "SGD",
          modelTree: "DecisionTree", modelRndForest: "RandomForest"}

In [40]:
outputs = []
for model, prefix in models.items():
    name=prefix+suffix
    outputs.append(other_tests(model, train_df, test_df, x_selection, submission_name=name, details=False))
outputs.sort(key=lambda x: x[0])

In [41]:
# DecisionTree_basicTest
# GaussianNB_basicTest
# KN_basicTest
# LinSVC_basicTest
# Percep_basicTest
# RandomForest_basicTest
# SGD_basicTest
# SVC_basicTest

In [179]:
#testset_acc = [0.62679, 0.73684, 0.55502, 0.63158, 0.65072, 0.75598, 0.64593, 0.62679]
#for i in range (len(testset_acc)):
#    test_acc =  "{:.2f}".format(testset_acc[i]*100)
#    outputs[i].insert(3, test_acc)
#    super_table.append(outputs[i])

In [77]:
make_table(super_table)
apply_theme('basic')

name,reg_function,trainset_acc,testset_acc,r2score,conf_matrx,kaggle_acc
logisticReg_basic,Survived = 3.99 + -0.85*Pclass + -0.00*Name + -2.35*Sex + -0.03*Age + -0.32*SibSp + -0.06*Parch + -0.00*Ticket + 0.01*Fare,79.97,nan,0.1696324007807416,"TN: 369, FP: 55, FN: 88, TP: 202",failed_evl
linearReg_basic,Survived = 1.40 + -0.19*Pclass + -0.00*Name + -0.49*Sex + -0.01*Age + -0.06*SibSp + -0.01*Parch + -0.00*Ticket + 0.00*Fare,40.25,nan,0.4025252345231439,nan,failed_evl
SVC_basic,nan,99.86,nan,0.9941932335718932,"TN: 424, FP: 0, FN: 1, TP: 289",failed_evl
KNC_basic,nan,82.49,nan,0.2741541964866623,"TN: 356, FP: 68, FN: 57, TP: 233",failed_evl
GaussianNB_basic,nan,78.57,nan,0.11156473649967458,"TN: 352, FP: 72, FN: 81, TP: 209",failed_evl
Perceptron_basic,nan,62.46,nan,-0.5562134027325961,"TN: 417, FP: 7, FN: 261, TP: 29",failed_evl
LinearSVC_basic,nan,49.44,nan,-1.0962426805465193,"TN: 94, FP: 330, FN: 31, TP: 259",failed_evl
SGDClassifier_basic,nan,65.69,nan,-0.422657774886142,"TN: 330, FP: 94, FN: 151, TP: 139",failed_evl
DecisionTreeClassifier_basic,nan,100.0,nan,1.0,"TN: 424, FP: 0, FN: 0, TP: 290",failed_evl
RandomForestClassifier_basic,nan,100.0,nan,1.0,"TN: 424, FP: 0, FN: 0, TP: 290",failed_evl


In [50]:
df = pd.DataFrame(super_table[1:], columns=super_table[0])
df.to_csv(r'C:/Users/ernest.chocholowski/Desktop/GIT/Titanic/table_eCh.csv', 
                      index=False)